<a href="https://colab.research.google.com/github/quantumRAV/16745_CourseProject/blob/main/PID_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2022-11-02 02:02:18 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [2]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  565.157 ms (2 allocations: 32.00 MiB)


In [ ]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


# **PID Control**


In [4]:
import Pkg
Pkg.instantiate()
Pkg.add("Plots")
Pkg.add("LinearAlgebra")
Pkg.add("ForwardDiff")
using LinearAlgebra, Plots
import ForwardDiff as FD
using JLD2
using Test
using Random

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed JpegTurbo_jll ──────────────── v3.0.2+0
   Installed Libmount_jll ───────────────── v2.39.3+0
   Installed GR_jll ─────────────────────── v0.73.3+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed LoggingExtras ──────────────── v1.0.3
   Installed ConcurrentUtilities ────────── v2.4.1
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed Measures ───────────────────── v0.3.2
   Installed Unitful ────────────────────── v1.19.0
   Installed Contour ────────────────────── v0.6.3
   Installed Grisu ──────────────────────── v1.0.2
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Plot

LoadError: ArgumentError: Package JLD2 not found in current path.
- Run `import Pkg; Pkg.add("JLD2")` to install the JLD2 package.

In [5]:
# Define parameters
mutable struct Params
    A::Matrix
    B::Matrix
    K::Float64
    Ki::Float64
    Kd::Float64
    prev_error::Vector
    target_position::Vector
end

# Koopman dynamics function, the equation, may need to add more
function koopman_dynamics(params::Params, z::Vector, u::Vector) #is u a vector? yes, up to the x y pressure data
    z_k_1 = params.A * z + params.B * u
    return z_k_1
end

# Mapping function from state x to lifted vector z (psi function)
function map_x_to_z(params::Params, x::Vector)
    return x # In this example, let's assume identity mapping
end

# Mapping function from lifted vector z to state x (zeta function)
function map_z_to_x(params::Params, z::Vector)
    return z # In this example, let's assume identity mapping
end

# PID control function , make this time, x_t is not being used here???
function PID(params::Params, z::Vector, x_t::Vector, v::Real)
    x = map_z_to_x(params, z)
    e = params.target_position - x_t #debugged
    v += e
    v = min(v_max, max(v_min, v))
    u_k = params.K * e + params.Ki * v  #note: sum(e) is that based on time or all 3 jaws' error at certain time t?
   # params.prev_error = e # PI controlled
    return u_k
end

# Simulation function
function simulation(params::Params, N::Int)
    x_sim = zeros(length(params.target_position), N)
    z_sim = zeros(length(params.target_position), N)
    u_sim = zeros(N-1)

    # Initial state
    x_sim[:, 1] .= params.target_position
    z_sim[:, 1] = map_x_to_z(params, x_sim[:, 1])

    for k in 1:N-1
        u_sim[k] = PID(params, z_sim[:, k], params.target_position[:,k])

        z_sim[:, k+1] = koopman_dynamics(params, z_sim[:, k], u_sim[k])
        x_sim[:, k+1] = map_z_to_x(params, z_sim[:, k+1])
    end

    return x_sim, u_sim
end

# Define parameters, I am not sure what to put here tbh (?)
nc = 3 #how many control inputs, x,y,pressure
ns = 3 #how many variable states, 3 jaws
A = [1 0 0; 0 1 0; 0 0 1]  #  A matrix, try with identity matrix A = [ns, ns]
B = [1 0 0; 0 1 0; 0 0 1]       #  B matrix, B = [ns, nc]
K = 1.0                 #  Proportional gain, P
Ki = 0.1                #  Integral gain, I
Kd = 0.01               #  Derivative gain, D
P_target_pressure = [1, 1, 1] #p_jaw1, p_jaw2, p_jaw3, matrix dim needs to be changed
#time VECTOR
#t = [ 0, 1/16, 2/16, ... 20] , t is element of real number
# P_target = P_jaw*sin(2*pi*f(t)ti), for i in t
params = Params(A, B, K, Ki, Kd, zeros(2), target_position)

# Run simulation
#fs= 16hz
N = 16 * 20 #per squence
x_sim, u_sim = simulation(params, N)

# Plotting
plot(x_sim[1, :], x_sim[2, :], label="Soft Grasper Trajectory", xlabel="X", ylabel="Y", legend=:topleft)
scatter!([params.target_position[1]], [params.target_position[2]], label="Target Position", markersize=10)


LoadError: UndefVarError: target_position not defined

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />